# Data Loading, Integration and Audit (DfT Road Safety 2024)

This notebook loads the 2024 UK Department for Transport road safety datasets, merges collision, and vehicle-level records, and constructs a clean dataset focused on young drivers (ages 17–25). The output will be used for downstream machine learning, fairness, and explainability analysis.


In [1]:
import sys
print(sys.executable)

/Users/sinjinisarkar/Desktop/3rd Year - CWKs/Semester 2/Individual Project/Projects/fyp-insurance-fairness-xai/.venv/bin/python


In [2]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

pd.set_option("display.max_columns", 100)

2.3.3
2.3.5


In [3]:
collisions = pd.read_csv("../data/raw/dft-road-casualty-statistics-collision-2024.csv")
vehicles = pd.read_csv("../data/raw/dft-road-casualty-statistics-vehicle-2024.csv")

print("Collisions shape:", collisions.shape)
print("Vehicles shape:", vehicles.shape)

/var/folders/j9/bfd315wn393_kw63y07n0s1m0000gn/T/ipykernel_35572/4206452099.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  collisions = pd.read_csv("../data/raw/dft-road-casualty-statistics-collision-2024.csv")


Collisions shape: (100927, 44)
Vehicles shape: (183514, 32)


/var/folders/j9/bfd315wn393_kw63y07n0s1m0000gn/T/ipykernel_35572/4206452099.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles = pd.read_csv("../data/raw/dft-road-casualty-statistics-vehicle-2024.csv")


In [4]:
collisions.head()

,collision_index,collision_year,collision_ref_no,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,collision_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,local_authority_highway_current,first_road_class,first_road_number,road_type,speed_limit,junction_detail_historic,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control_historic,pedestrian_crossing_physical_facilities_historic,pedestrian_crossing,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards_historic,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,enhanced_severity_collision,collision_injury_based,collision_adjusted_severity_serious,collision_adjusted_severity_slight
0,202417M119024,2024,17M119024,450057,519938,-1.22722,54.57219,17,3,2,1,05/12/2024,5,16:10,-1,E06000002,E06000002,E06000002,6,0,6,30,-1,0,-1,-1,-1,-1,-1,0,4,2,2,-1,-1,0,1,3,2,E01035190,3,1,0.000000,1.000000
1,202417S312124,2024,17S312124,445858,516830,-1.29265,54.54466,17,3,1,1,22/10/2024,3,14:56,-1,E06000004,E06000004,E06000004,6,0,6,30,-1,0,-1,-1,-1,-1,-1,0,1,1,1,-1,-1,0,1,3,2,E01012239,3,1,0.000000,1.000000
2,2024070110901,2024,070110901,364765,390979,-2.53157,53.41443,7,3,2,1,24/01/2024,4,17:50,-1,E06000007,E06000007,E06000007,6,0,6,30,3,13,4,6,0,0,0,0,4,1,1,0,0,0,2,2,2,E01012459,-1,0,0.014743,0.985257
3,2024041446676,2024,041446676,365989,419804,-2.51630,53.67359,4,2,1,1,22/05/2024,4,17:45,-1,E06000008,E06000008,E06000008,3,675,6,50,-1,0,-1,-1,-1,-1,-1,0,1,5,5,-1,-1,0,2,1,2,E01012628,6,1,1.000000,0.000000
4,2024041478641,2024,041478641,366864,430071,-2.50416,53.76592,4,2,2,1,11/08/2024,1,15:36,-1,E06000008,E06000008,E06000008,3,6119,3,50,-1,16,4,6,0,-1,-1,0,1,1,1,-1,-1,0,1,1,2,E01012581,7,1,1.000000,0.000000


In [5]:
vehicles.head()

,collision_index,collision_year,collision_ref_no,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre_historic,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane_historic,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver_historic,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,lsoa_of_driver,escooter_flag,driver_distance_banding
0,2024471526976,2024,471526976,2,4,0,-1,19,4,8,-1,0,8,0,0,0,0,3,1,-1,9,1,23,5,234,1,16,MAKE AND MODEL REDACTED,8,E01020954,0,1
1,2024471493223,2024,471493223,2,9,0,-1,2,0,0,-1,0,0,0,0,0,0,1,1,-1,6,-1,-1,-1,2198,1,15,MAKE AND MODEL REDACTED,-1,-1,0,-1
2,2024010504128,2024,010504128,2,9,0,2,2,0,0,0,0,0,0,0,0,0,2,1,6,6,3,-1,-1,2198,1,18,MAKE AND MODEL REDACTED,-1,-1,0,-1
3,2024302400896,2024,302400896,2,9,0,18,19,1,4,0,0,0,0,0,0,0,1,1,6,6,1,56,9,6496,1,0,MAKE AND MODEL REDACTED,5,E01019782,0,2
4,2024631472488,2024,631472488,2,90,0,-1,19,7,3,-1,0,8,0,0,0,0,4,1,-1,9,1,55,8,2184,2,0,MAKE AND MODEL REDACTED,7,W01000623,0,4


In [6]:
set(collisions.columns).intersection(set(vehicles.columns))

{'collision_index', 'collision_ref_no', 'collision_year'}

### Data Integration Rationale

The DfT road safety data is provided at multiple levels of granularity. The *collision* dataset contains one record per accident, while the *vehicle* dataset contains one record per vehicle involved in an accident.

Since a single collision may involve multiple vehicles, the relationship between the vehicle-level data and the collision-level data is **many-to-one** (vehicles → collision).

The datasets are merged using `collision_index` as the join key, producing a vehicle-level dataset enriched with accident context. This structure closely mirrors real-world insurance risk data, where risk is assessed per vehicle or driver, but informed by shared accident conditions.

The merge is explicitly validated using `validate="many_to_one"` to ensure data integrity and prevent unintended duplication.

In [7]:
df = vehicles.merge(
    collisions,
    on="collision_index",
    how="inner",
    validate="many_to_one"
)

df.shape

(88811, 75)

In [8]:
# Compute the proportion of missing values per column to audit data completeness
df.isna().mean().sort_values(ascending=False).head(10)

local_authority_highway_current    0.000068
collision_index                    0.000000
second_road_class                  0.000000
junction_detail                    0.000000
junction_detail_historic           0.000000
speed_limit                        0.000000
road_type                          0.000000
first_road_number                  0.000000
first_road_class                   0.000000
local_authority_highway            0.000000
dtype: float64

### Verifying the relationship
So, we have 49,152 unique collisions and 88,811 vehicle rows. So on average: ~1.8 vehicles per collision, which proves many to one relationship

In [9]:
# Compare the number of unique collisions to total rows to confirm vehicle-level granularity
df["collision_index"].nunique(), len(df)

(49152, 88811)

In [10]:
# Inspect the first few rows of the merged dataset to verify successful integration
df.head()

,collision_index,collision_year_x,collision_ref_no_x,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre_historic,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane_historic,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver_historic,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,lsoa_of_driver,escooter_flag,driver_distance_banding,collision_year_y,collision_ref_no_y,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,collision_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,local_authority_highway_current,first_road_class,first_road_number,road_type,speed_limit,junction_detail_historic,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control_historic,pedestrian_crossing_physical_facilities_historic,pedestrian_crossing,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards_historic,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,enhanced_severity_collision,collision_injury_based,collision_adjusted_severity_serious,collision_adjusted_severity_slight
0,2024631472488,2024,631472488,2,90,0,-1,19,7,3,-1,0,8,0,0,0,0,4,1,-1,9,1,55,8,2184,2,0,MAKE AND MODEL REDACTED,7,W01000623,0,4,2024,631472488,235423,251976,-4.40642,52.14149,63,3,2,1,26/07/2024,6,15:00,-1,W06000008,W06000008,W06000008,3,487,6,60,-1,16,3,4,4321,-1,-1,0,1,1,1,-1,-1,0,2,1,-1,W01000545,3,1,0.0,1.0
1,2024401457067,2024,401457067,1,9,0,-1,6,7,3,-1,0,0,0,0,0,0,1,1,-1,6,1,26,6,1685,2,9,HYUNDAI TUCSON,10,E01017611,0,3,2024,401457067,505407,222089,-0.46983,51.88740,40,2,2,1,05/06/2024,4,17:43,-1,E06000056,E06000056,E06000056,3,5065,3,50,-1,0,-1,6,0,-1,-1,0,1,1,1,-1,-1,0,2,1,2,E01017562,7,1,1.0,0.0
2,2024031497068,2024,031497068,1,9,0,-1,19,1,5,-1,0,0,5,0,1,4,1,1,-1,9,1,48,8,1598,1,3,HYUNDAI TUCSON,1,-1,0,5,2024,031497068,343504,550871,-2.88140,54.84944,3,3,1,2,28/09/2024,7,11:50,-1,E06000063,E06000063,E06000063,1,6,3,70,-1,0,-1,-1,-1,-1,-1,0,1,1,1,-1,-1,0,2,1,1,E01034074,3,1,0.0,1.0
3,2024061401609,2024,061401609,1,9,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,0,-1,9,-1,2,2,52,8,1598,8,1,HYUNDAI TUCSON,-1,-1,0,-1,2024,061401609,385634,390809,-2.21760,53.41389,6,2,1,1,16/01/2024,3,15:10,-1,E08000003,E08000003,E08000003,3,34,3,40,-1,0,-1,-1,-1,-1,-1,-1,1,1,2,-1,-1,-1,1,3,2,E01005168,7,1,1.0,0.0
4,2024061470881,2024,061470881,2,9,0,-1,9,6,3,-1,0,8,0,0,0,0,3,1,-1,9,1,48,8,1591,1,6,HYUNDAI TUCSON,-1,-1,0,-1,2024,061470881,395408,397510,-2.07065,53.47430,6,2,2,2,22/07/2024,2,17:10,-1,E08000008,E08000008,E08000008,6,0,6,20,-1,13,4,4,6175,-1,-1,0,1,1,1,-1,-1,0,1,1,2,E01006021,7,1,1.0,0.0


### Filter to Young Drivers (Ages 17–25)
This project focuses specifically on young drivers, defined as individuals aged between 17 and 25 inclusive. This age range is widely used in UK road safety and insurance literature to represent drivers with limited driving experience and elevated insurance risk.

Filtering to this subgroup at an early stage ensures that subsequent modelling, fairness analysis, and explainability techniques are applied to a clearly defined and relevant population, rather than the general driving population.

In [11]:
# Filter dataset to young drivers aged 17–25 inclusive
df_young = df[
    (df["age_of_driver"] >= 17) & 
    (df["age_of_driver"] <= 25)
].copy()

# Check resulting dataset size
df_young.shape

(13709, 75)

So, we have 13,709 young-driver vehicles after filtering

In [12]:
# Verify age range after filtering
df_young["age_of_driver"].describe()

count    13709.000000
mean        21.149537
std          2.535299
min         17.000000
25%         19.000000
50%         21.000000
75%         23.000000
max         25.000000
Name: age_of_driver, dtype: float64

In [13]:
# Save young driver dataset for downstream modelling
df_young.to_csv(
    "../data/processed/df_young_drivers_2024.csv",
    index=False
)

### Define Target Variable (Insurance Risk Proxy)
We do not have access to proprietary insurer pricing or claims datasets. To model insurance-relevant “risk” using public UK road safety data, we define a proxy target variable based on collision outcome severity. Specifically, we treat Fatal and Serious collisions as higher-risk events, and Slight collisions as lower-risk events. This yields a binary classification problem suitable for baseline modelling, fairness auditing, and explainability.


In [14]:
df_young.columns.tolist()

['collision_index',
 'collision_year_x',
 'collision_ref_no_x',
 'vehicle_reference',
 'vehicle_type',
 'towing_and_articulation',
 'vehicle_manoeuvre_historic',
 'vehicle_manoeuvre',
 'vehicle_direction_from',
 'vehicle_direction_to',
 'vehicle_location_restricted_lane_historic',
 'vehicle_location_restricted_lane',
 'junction_location',
 'skidding_and_overturning',
 'hit_object_in_carriageway',
 'vehicle_leaving_carriageway',
 'hit_object_off_carriageway',
 'first_point_of_impact',
 'vehicle_left_hand_drive',
 'journey_purpose_of_driver_historic',
 'journey_purpose_of_driver',
 'sex_of_driver',
 'age_of_driver',
 'age_band_of_driver',
 'engine_capacity_cc',
 'propulsion_code',
 'age_of_vehicle',
 'generic_make_model',
 'driver_imd_decile',
 'lsoa_of_driver',
 'escooter_flag',
 'driver_distance_banding',
 'collision_year_y',
 'collision_ref_no_y',
 'location_easting_osgr',
 'location_northing_osgr',
 'longitude',
 'latitude',
 'police_force',
 'collision_severity',
 'number_of_vehicle

### Collision Severity Coding

The `collision_severity` variable follows the official STATS19 coding scheme defined by the UK Department for Transport:

- 1 = Fatal  
- 2 = Serious  
- 3 = Slight  

This mapping is taken from the *Road Safety Open Data Guide (2024)* and is used consistently throughout this project.

In [15]:
df_young["collision_severity"].value_counts(dropna=False)

collision_severity
3    10090
2     3404
1      215
Name: count, dtype: int64

The total is 13709 same as the column no of our processed dataset and hence this tells us that there is no missing outcome labels and no data leakage or corruption

In [16]:
# Define binary insurance risk target
# High risk: Fatal (1) or Serious (2)
# Low risk: Slight (3)

df_young["high_risk"] = df_young["collision_severity"].isin([1, 2]).astype(int)

# Inspect class distribution
df_young["high_risk"].value_counts(normalize=True)

high_risk
0    0.736013
1    0.263987
Name: proportion, dtype: float64

In [17]:
# Define binary insurance risk target
# High risk: Fatal (1) or Serious (2)
# Low risk: Slight (3)
df_young["high_risk"] = df_young["collision_severity"].isin([1, 2]).astype(int)


In [18]:
# Inspect class distribution
df_young["high_risk"].value_counts()


high_risk
0    10090
1     3619
Name: count, dtype: int64

In [19]:
df_young["high_risk"].value_counts(normalize=True)

high_risk
0    0.736013
1    0.263987
Name: proportion, dtype: float64

In [20]:
# Cross-check mapping between original severity and binary target
pd.crosstab(
    df_young["collision_severity"],
    df_young["high_risk"],
    rownames=["collision_severity"],
    colnames=["high_risk"]
)

high_risk,0,1
collision_severity,,
1,0,215
2,0,3404
3,10090,0


The binary risk variable was validated using a cross-tabulation against the original severity labels, confirming a one-to-one mapping.

In [21]:
df_young.groupby("high_risk")["age_of_driver"].describe()

,count,mean,std,min,25%,50%,75%,max
high_risk,,,,,,,,
0,10090.0,21.209812,2.533722,17.0,19.0,21.0,24.0,25.0
1,3619.0,20.981487,2.532473,17.0,19.0,21.0,23.0,25.0


Even within the young-driver group, age alone does not clearly separate risk, motivating the use of multivariate models and fairness-aware analysis.

### First Protected attribute: Sex of driver

The `sex_of_driver` variable follows the official STATS19 coding scheme defined by the UK Department for Transport:

- 1 = Male  
- 2 = Female
- 3 = Not known
- -1 = Data missing or out of range

This mapping is taken from the *Road Safety Open Data Guide (2024)* and is used consistently throughout this project.

In [22]:
df_young["sex_of_driver"].value_counts(dropna=False)

sex_of_driver
 1    9613
 2    3763
 3     278
-1      55
Name: count, dtype: int64

In [23]:
# Clean protected attribute: sex_of_driver
# Keep only valid, known categories for fairness analysis
df_young_fair = df_young[
    df_young["sex_of_driver"].isin([1, 2])
].copy()

df_young_fair["sex_of_driver"].value_counts()

sex_of_driver
1    9613
2    3763
Name: count, dtype: int64

Protected attributes were audited prior to fairness evaluation. Records with missing or indeterminate subgroup labels were excluded from fairness analysis to ensure valid subgroup comparisons.

### Second Protected attribute: Age of driver


In [24]:
df_young["age_band_of_driver"].value_counts(dropna=False)

age_band_of_driver
5    7670
4    6039
Name: count, dtype: int64

In [25]:
# Keep only rows with valid target + protected attributes
# (sex_of_driver: keep 1=Male, 2=Female; drop 3=Not known and -1 missing/out of range)
df_young_fair = df_young[
    df_young["high_risk"].notna() &
    df_young["sex_of_driver"].isin([1, 2]) &
    df_young["age_band_of_driver"].notna()
].copy()

df_young_fair.shape

(13376, 76)

In [27]:
df_young_fair["sex_of_driver"].value_counts(dropna=False)


sex_of_driver
1    9613
2    3763
Name: count, dtype: int64

In [28]:
df_young_fair["age_band_of_driver"].value_counts(dropna=False)


age_band_of_driver
5    7517
4    5859
Name: count, dtype: int64

In [29]:
df_young_fair["high_risk"].value_counts(normalize=True)

high_risk
0    0.734973
1    0.265027
Name: proportion, dtype: float64

In [30]:
df_young_fair.to_csv(
    "../data/processed/df_young_fair_2024.csv",
    index=False
)

In [31]:
df_young_fair.columns.tolist()

['collision_index',
 'collision_year_x',
 'collision_ref_no_x',
 'vehicle_reference',
 'vehicle_type',
 'towing_and_articulation',
 'vehicle_manoeuvre_historic',
 'vehicle_manoeuvre',
 'vehicle_direction_from',
 'vehicle_direction_to',
 'vehicle_location_restricted_lane_historic',
 'vehicle_location_restricted_lane',
 'junction_location',
 'skidding_and_overturning',
 'hit_object_in_carriageway',
 'vehicle_leaving_carriageway',
 'hit_object_off_carriageway',
 'first_point_of_impact',
 'vehicle_left_hand_drive',
 'journey_purpose_of_driver_historic',
 'journey_purpose_of_driver',
 'sex_of_driver',
 'age_of_driver',
 'age_band_of_driver',
 'engine_capacity_cc',
 'propulsion_code',
 'age_of_vehicle',
 'generic_make_model',
 'driver_imd_decile',
 'lsoa_of_driver',
 'escooter_flag',
 'driver_distance_banding',
 'collision_year_y',
 'collision_ref_no_y',
 'location_easting_osgr',
 'location_northing_osgr',
 'longitude',
 'latitude',
 'police_force',
 'collision_severity',
 'number_of_vehicle